In [2]:
import librosa
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Argumentations:
 - Half tone pitch shifting
 - One tone pitch shifting
 - Time overlap
 - Noise injection
 - Each combination (Tone pitch either or)

In [3]:
def pitch_shift(data, sampling_rate, pitch_factor):
    return librosa.effects.pitch_shift(data, sampling_rate, pitch_factor)


def add_noise(data, noise_factor): # 0.005
    noise = np.random.randn(len(data))
    augmented_data = data + noise_factor * noise
    augmented_data = augmented_data.astype(type(data[0])) # Cast back to same data type
    return augmented_data


def shift_roll(data):
    return np.roll(data, 1600)

